<a href="https://colab.research.google.com/github/hattab-abdessalam/deep/blob/main/Iris_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob as gb
import shutil
import cv2
import glob
import numpy as np
import keras
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm_notebook
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import load_model, Model
from sklearn.svm import SVC
from keras_preprocessing import image


In [3]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input

In [4]:
IMG_SHAPE = (224, 224,3)
base_model =  Xception(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.summary()

83689472/83683744 [==============================] - 0s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________

In [5]:
for layer in base_model.layers[:]:
  layer.trainable = False
for i, layer in enumerate(base_model.layers):
  print(i, layer.name, "-", layer.trainable)

0 input_1 - False
1 block1_conv1 - False
2 block1_conv1_bn - False
3 block1_conv1_act - False
4 block1_conv2 - False
5 block1_conv2_bn - False
6 block1_conv2_act - False
7 block2_sepconv1 - False
8 block2_sepconv1_bn - False
9 block2_sepconv2_act - False
10 block2_sepconv2 - False
11 block2_sepconv2_bn - False
12 conv2d - False
13 block2_pool - False
14 batch_normalization - False
15 add - False
16 block3_sepconv1_act - False
17 block3_sepconv1 - False
18 block3_sepconv1_bn - False
19 block3_sepconv2_act - False
20 block3_sepconv2 - False
21 block3_sepconv2_bn - False
22 conv2d_1 - False
23 block3_pool - False
24 batch_normalization_1 - False
25 add_1 - False
26 block4_sepconv1_act - False
27 block4_sepconv1 - False
28 block4_sepconv1_bn - False
29 block4_sepconv2_act - False
30 block4_sepconv2 - False
31 block4_sepconv2_bn - False
32 conv2d_2 - False
33 block4_pool - False
34 batch_normalization_2 - False
35 add_2 - False
36 block5_sepconv1_act - False
37 block5_sepconv1 - False
38 bl

In [6]:
x = Flatten()(base_model.output)
y = Dense(1000, activation='relu')(x)
prediction = Dense(108,activation='softmax')(y)
model = Model(inputs=base_model.input, outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [7]:
model.compile(  
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics='accuracy')

In [8]:
data_path='/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2'

In [9]:
train_dir = os.path.join(data_path, "train")
test_dir = os.path.join(data_path, "test")
# useful for getting number of classes
folders = glob(train_dir+'/*')
folders

['/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/1',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/107',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/10',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/102',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/100',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/101',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/104',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/105',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/106',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/103',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/108',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/11',
 '/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/12',
 '/content/drive/

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                  #  rotation_range=30,
                                  #   width_shift_range=0.1,
                                  #   height_shift_range=0.1,
                                  #   #channel_shift_range=10.,
                             
                                  #   fill_mode='nearest'
                                   )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 540 images belonging to 108 classes.
Found 216 images belonging to 108 classes.


In [11]:
mcp1_save = keras.callbacks.ModelCheckpoint('mdl1_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp2_save = keras.callbacks.ModelCheckpoint('mdl2_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
early=keras.callbacks.EarlyStopping(monitor="val_accuracy",min_delta=0,patience=50,verbose=1,mode='max')
callbacklist=[mcp1_save,mcp2_save,early]
ThisModel = model.fit(training_set, epochs=300 , batch_size=100,verbose=1, validation_data=test_set,callbacks=callbacklist)

Epoch 1/300
17/17 [==============================] - 207s 10s/step - loss: 48.8826 - accuracy: 0.0280 - val_loss: 23.5605 - val_accuracy: 0.1389
Epoch 2/300
17/17 [==============================] - 12s 460ms/step - loss: 17.1706 - accuracy: 0.1768 - val_loss: 7.5568 - val_accuracy: 0.2778
Epoch 3/300
17/17 [==============================] - 12s 687ms/step - loss: 4.4464 - accuracy: 0.3797 - val_loss: 3.3689 - val_accuracy: 0.3056
Epoch 4/300
17/17 [==============================] - 8s 481ms/step - loss: 1.8480 - accuracy: 0.5187 - val_loss: 2.2886 - val_accuracy: 0.5185
Epoch 5/300
17/17 [==============================] - 8s 472ms/step - loss: 1.1322 - accuracy: 0.6987 - val_loss: 1.7480 - val_accuracy: 0.6296
Epoch 6/300
17/17 [==============================] - 8s 473ms/step - loss: 0.6712 - accuracy: 0.8182 - val_loss: 1.7124 - val_accuracy: 0.6343
Epoch 7/300
17/17 [==============================] - 8s 475ms/step - loss: 0.5463 - accuracy: 0.8534 - val_loss: 1.3396 - val_accuracy: 0

In [12]:

model.load_weights('mdl1_wts.hdf5')
model.evaluate(test_set)

7/7 [==============================] - 1s 137ms/step - loss: 2.2612 - accuracy: 0.8056


[2.261152982711792, 0.8055555820465088]

In [13]:
layer_name = 'block14_sepconv2_act'
extract_feature_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
extract_feature_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [14]:
X_train = np.zeros(( 108*5,7*7*2048))
y_train = np.zeros(108*5)
trainpath="/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/train/"
i=0
for folder in  os.listdir(trainpath ) : 
    folderclasspath =  trainpath + folder 
    print(folder)
    files = gb.glob(pathname= str( folderclasspath + '/*'))
    for file in files: 
        images = image.load_img(file, target_size=(224, 224))    
        x = image.img_to_array(images)
        x = np.expand_dims(x, axis=0)
        x = x/255.0    
        X_train[i]= np.asanyarray(extract_feature_model.predict(x)).flatten()
        y_train[i]=(int(folder)-1)
        i=i+1

testpath="/content/drive/MyDrive/datasets/iris data/images/casia_v1_1_2/test/"

X_test = np.zeros((108*2,7*7*2048))
y_test = np.zeros(108*2)
i=0
for folder in  os.listdir(testpath ) : 
    folderclasspath =  testpath + folder 
    print(folder)
    files = gb.glob(pathname= str( folderclasspath + '/*'))
    for file in files: 
        images = image.load_img(file, target_size=(224, 224))    
        x = image.img_to_array(images)
        x = np.expand_dims(x, axis=0)
        x = x/255.0    
        X_test[i]= np.asanyarray(extract_feature_model.predict(x)).flatten()
        y_test[i]=(int(folder)-1)
        i=i+1


1
107
10
102
100
101
104
105
106
103
108
11
12
16
20
2
14
18
17
15
19
13
21
22
28
26
24
29
25
3
27
23
30
31
32
39
37
33
40
4
36
38
34
35
41
44
46
42
45
43
49
48
47
5
50
51
54
56
53
52
58
59
57
55
6
60
61
62
64
70
66
63
69
7
65
68
67
71
72
73
74
79
78
8
76
80
77
75
81
82
91
88
87
89
9
84
90
85
83
86
92
93
94
98
99
96
95
97
100
10
1
101
12
102
105
107
108
11
103
104
106
13
17
21
18
20
16
15
2
14
19
22
23
24
28
26
25
30
27
31
3
32
29
33
34
35
43
39
37
40
4
41
36
38
42
47
45
5
51
49
50
46
48
52
44
55
58
53
59
57
6
56
54
60
61
62
66
68
69
7
70
63
67
64
65
71
75
73
78
79
8
77
72
74
76
80
81
82
9
90
89
84
87
86
83
85
88
91
92
98
95
93
97
94
99
96


In [ ]:
clf = SVC(kernel='linear', random_state=42)
clf.fit(X_train, y_train)

In [ ]:

accuracy = clf.score(X_test, y_test)
print("\nConvNetSVM accuracy =", accuracy)

In [ ]:
import cv2
img = cv2.imread('C:Users/BS/Documents/iris_recognition/1.bmp',0)  # this is a PIL image
blur = cv2.bilateralFilter(img,9,75,75)

error: ignored

In [ ]:
import cv2
import numpy as np
import cv2

def rotate_image(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))
def flip (img,n):
    image = img[:, :int(img.shape[1] * n)]
    image = cv2.flip(image, 1)
    image = np.concatenate((image, img), axis=1)
    image = image[:, :img.shape[1]]
    return image

def data_augmentations(image_path):
    print(image_path)
    img = cv2.imread(image_path,0)  # this is a PIL image
    print(img)

    blur = cv2.bilateralFilter(img,9,75,75)
    print('augmentation/'+str(image_path.split('/')[8])+'/b_' + str(image_path.split('/')[9]))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/b_' + str(image_path.split('/')[9]), blur)
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r75_' + str(image_path.split('/')[9]), rotate_image(img, 75))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r150_' + str(image_path.split('/')[9]), rotate_image(img, 150))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r225_' + str(image_path.split('/')[9]), rotate_image(img, 225))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r4_' + str(image_path.split('/')[9]), flip(img, 1/4))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r2_' + str(image_path.split('/')[9]), flip(img, 1/2))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r34_' + str(image_path.split('/')[9]), flip(img, 3/4))
    cv2.imwrite('/content/drive/My Drive/datasets/iris data/augmentation/'+str(image_path.split('/')[8])+'/r1_' + str(image_path.split('/')[9]), flip(img, 1))

testpath="/content/drive/My Drive/datasets/iris data/casia_v1/train/"
import os
import glob as gb
for folder in  os.listdir(testpath ) :

    folderclasspath =  testpath + folder
    os.mkdir('/content/drive/My Drive/datasets/iris data/augmentation/' + folder)
    print(folder)
    files = gb.glob(pathname= str( folderclasspath + '/*'))
    for file in files:
        data_augmentations(file)

97
/content/drive/My Drive/datasets/iris data/casia_v1/train/97/097_1_3.jpg
[[  7   0   3 ...   0   4  10]
 [161  16  12 ...  16  15 159]
 [162  17  17 ...  10  11 162]
 ...
 [  3   2  57 ...  71  18  14]
 [ 91  54  46 ...  47 195 162]
 [ 62  61  88 ...  44 197 131]]
augmentation/97/b_097_1_3.jpg
/content/drive/My Drive/datasets/iris data/casia_v1/train/97/097_2_1.jpg
[[ 20  26  22 ...   0   1   0]
 [ 28  28  25 ...   1   0   4]
 [ 27  28  23 ...   0   0   0]
 ...
 [ 53 200 196 ... 139 101 121]
 [ 14 124   6 ... 120 155  55]
 [ 53  64 126 ... 157  72 121]]
augmentation/97/b_097_2_1.jpg
/content/drive/My Drive/datasets/iris data/casia_v1/train/97/097_2_3.jpg
[[ 13  19  14 ...   0   4   4]
 [ 20  12  12 ...  21  19  15]
 [ 20  27  16 ...  20  27  25]
 ...
 [141 242 171 ...  91 175 195]
 [200 167 237 ... 251 199 232]
 [196 254 238 ... 226 249 234]]
augmentation/97/b_097_2_3.jpg
/content/drive/My Drive/datasets/iris data/casia_v1/train/97/097_2_2.jpg
[[  0   4   7 ...   9   5   2]
 [211 17


ConvNetSVM accuracy = 0.8616071428571429
